## 기본 예시: 프롬프트 + 모델 + 출력 파서

가장 기본적이고 일반적인 사용 사례는 prompt 템플릿과 모델을 함께 연결하는 것입니다. 이것이 어떻게 작동하는지 보기 위해, 각 나라별 수도를 물어보는 Chain을 생성해 보겠습니다.


In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH01-Basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


## 프롬프트 템플릿의 활용

`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다
- 사용법
  - `template`: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 `{}`는 변수를 나타냅니다.
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.

In [3]:
from langchain_teddynote.messages import stream_response  # 스트리밍 출력
from langchain_core.prompts import PromptTemplate

`from_template()` 메소드를 사용하여 PromptTemplate 객체 생성


In [4]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [5]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [6]:
# prompt 생성
prompt = prompt_template.format(country="미국")
prompt

'미국의 수도는 어디인가요?'

In [7]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
)

## Chain 생성

### LCEL(LangChain Expression Language)

![lcel.png](./images/lcel.png)

여기서 우리는 LCEL을 사용하여 다양한 구성 요소를 단일 체인으로 결합합니다

```
chain = prompt | model | output_parser
```

`|` 기호는 [unix 파이프 연산자](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>)와 유사하며, 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달합니다.

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달됩니다. 각 구성 요소를 개별적으로 살펴보면 무슨 일이 일어나고 있는지 이해할 수 있습니다.


In [8]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 설명해주세요.")

model = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

chain = prompt | model

In [29]:
chain

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='{topic} 에 대해 설명해주세요.')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x1553a0c50>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x15534f910>, root_client=<openai.OpenAI object at 0x1553dbbd0>, root_async_client=<openai.AsyncOpenAI object at 0x155398a90>, model_name='gpt-4o-mini', temperature=0.1, model_kwargs={}, openai_api_key=SecretStr('**********'))

### invoke() 호출

- python 딕셔너리 형태로 입력값을 전달합니다.(키: 값)
- invoke() 함수 호출 시, 입력값을 전달합니다.

In [30]:
# input 딕셔너리에 주제를 '인공지능 모델의 학습 원리'으로 설정합니다.
input = {
    "topic": "인공지능 모델의 학습 원리",
}

In [20]:
# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달합니다.
# 이를 통해 AI 모델이 생성한 메시지를 반환합니다.
chain.invoke(input)

AIMessage(content='좋아! 인공지능 모델의 학습 원리를 아주 쉽게 설명해볼게.\n\n인공지능은 마치 똑똑한 로봇 친구 같아. 이 로봇 친구는 많은 그림과 이야기를 보고 배우면서 점점 더 똑똑해져. \n\n예를 들어, 너가 사과와 바나나를 가르쳐준다고 생각해보자. 처음에는 로봇 친구가 사과와 바나나를 잘 모르겠지. 그래서 너가 사과를 보여주면서 "이건 사과야!"라고 말해주면, 로봇 친구는 그걸 기억해. 그리고 바나나를 보여주면서 "이건 바나나야!"라고 말해주면, 또 기억해.\n\n이렇게 여러 번 반복해서 가르쳐주면, 로봇 친구는 사과와 바나나를 구별할 수 있게 돼. 나중에는 새로운 사과나 바나나를 보여줘도 잘 알아볼 수 있어!\n\n그래서 인공지능은 많은 정보를 보고 배우면서 점점 더 똑똑해지는 거야. 마치 우리가 학교에서 배우는 것과 비슷해!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 251, 'prompt_tokens': 33, 'total_tokens': 284, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-573d0b39-ae60-493e-b2c3-3cfb860fd20d-0', usage_metadata={'input_tokens': 33, 'output_tokens'

아래는 스트리밍을 출력하는 예시 입니다.

In [9]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

`<bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x107c1d990>>`는 IPython 커널의 `raw_input` 메서드에 대한 바운드 메서드 객체를 나타냅니다. 이 객체는 IPython 환경에서 사용자 입력을 받을 수 있는 메서드입니다.

### 구성 요소 설명:

1. **bound method**: "bound method"는 특정 객체(instance)에 바인딩된 메서드를 의미합니다. 즉, 이 메서드는 특정 클래스의 인스턴스와 연결되어 있으며, 그 인스턴스의 상태에 접근할 수 있습니다.

2. **Kernel**: `Kernel`은 IPython의 커널을 나타내는 클래스입니다. 커널은 코드 실행, 입력 및 출력 처리, 그리고 클라이언트와의 통신을 담당합니다.

3. **raw_input**: 이 메서드는 사용자로부터 입력을 받을 때 사용됩니다. Python 2에서는 `raw_input()`이라는 이름으로 존재했지만, Python 3에서는 `input()`으로 변경되었습니다. IPython에서는 이 메서드를 통해 사용자에게 입력을 요청할 수 있습니다.

4. **<ipykernel.ipkernel.IPythonKernel object at 0x107c1d990>**: 이 부분은 `IPythonKernel` 클래스의 인스턴스를 나타내며, 메모리 주소 `0x107c1d990`에서 위치하고 있음을 보여줍니다. 이 객체는 IPython 커널의 기능을 수행합니다.

### 요약:
이 표현은 IPython 커널의 `raw_input` 메서드가 특정 커널 인스턴스에 바인딩되어 있음을 나타내며, 이를 통해 사용자가 입력을 받을 수 있는 기능을 제공합니다. IPython 환경에서 사용자와 상호작용할 때 유용하게 사용됩니다.

### 출력파서(Output Parser)


In [32]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

Chain 에 출력파서를 추가합니다.

In [33]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser

In [34]:
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"topic": "인공지능 모델의 학습 원리"}
chain.invoke(input)

'인공지능 모델의 학습 원리는 주로 기계 학습(Machine Learning)과 딥러닝(Deep Learning)이라는 두 가지 주요 분야로 나눌 수 있습니다. 이들 각각의 원리는 다음과 같습니다.\n\n### 1. 기계 학습 (Machine Learning)\n\n기계 학습은 데이터에서 패턴을 학습하여 예측이나 결정을 내리는 알고리즘을 개발하는 분야입니다. 기계 학습의 기본 원리는 다음과 같습니다:\n\n- **데이터 수집**: 모델을 학습시키기 위해 필요한 데이터를 수집합니다. 이 데이터는 입력(features)과 출력(labels)으로 구성됩니다.\n\n- **모델 선택**: 문제에 적합한 알고리즘(예: 회귀, 분류, 군집화 등)을 선택합니다.\n\n- **훈련(Training)**: 선택한 모델에 데이터를 입력하여 학습을 진행합니다. 이 과정에서 모델은 입력 데이터와 출력 데이터 간의 관계를 학습합니다.\n\n- **손실 함수(Loss Function)**: 모델의 예측값과 실제값 간의 차이를 측정하는 함수입니다. 모델의 성능을 평가하고 개선하기 위해 사용됩니다.\n\n- **최적화(Optimization)**: 손실 함수를 최소화하기 위해 모델의 파라미터를 조정합니다. 일반적으로 경사 하강법(Gradient Descent)과 같은 최적화 알고리즘이 사용됩니다.\n\n- **검증(Validation)**: 학습된 모델의 성능을 평가하기 위해 별도의 검증 데이터를 사용합니다. 이를 통해 모델이 과적합(overfitting)되지 않도록 합니다.\n\n### 2. 딥러닝 (Deep Learning)\n\n딥러닝은 인공 신경망(Artificial Neural Networks)을 기반으로 한 기계 학습의 한 분야로, 특히 대량의 데이터와 복잡한 패턴을 처리하는 데 강력한 성능을 발휘합니다. 딥러닝의 학습 원리는 다음과 같습니다:\n\n- **신경망 구조**: 여러 층(layer)으로 구성된 인공 신경망을 사용합니다. 각 층은 노드(node)로 구성되며, 노드 간

In [ ]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

### 템플릿을 변경하여 적용

- 아래의 프롬프트 내용을 얼마든지 **변경** 하여 테스트 해볼 수 있습니다.
- `model_name` 역시 변경하여 테스트가 가능합니다.

In [35]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗모델을 초기화합니다.
model = ChatOpenAI(model_name="gpt-4o-mini")

# 문자열 출력 파서를 초기화합니다.
output_parser = StrOutputParser()

In [36]:
# 체인을 구성합니다.
chain = prompt | model | output_parser

In [37]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

- 영어 회화:
  - Waiter: Good evening! Welcome to our restaurant. How many people are in your party?
  - You: Good evening! Just one, please.
  - Waiter: Great! Here’s the menu. Can I get you something to drink while you decide?
  - You: Yes, I’d like a glass of water, please.
  - Waiter: Of course! Are you ready to order, or do you need a few more minutes?
  - You: I think I’m ready. I’d like the grilled chicken salad, please.
  - Waiter: Excellent choice! Would you like any dressing with that?
  - You: Yes, please. I’ll have the balsamic vinaigrette.
  - Waiter: Perfect! I’ll get that order in for you.

- 한글 해석:
  - 웨이터: 좋은 저녁입니다! 저희 식당에 오신 것을 환영합니다. 몇 분이세요?
  - 당신: 좋은 저녁입니다! 혼자입니다.
  - 웨이터: 좋습니다! 여기 메뉴입니다. 결정하시는 동안 음료수 드릴까요?
  - 당신: 네, 물 한 잔 주세요.
  - 웨이터: 물론입니다! 주문할 준비가 되셨나요, 아니면 좀 더 시간이 필요하신가요?
  - 당신: 이제 준비된 것 같아요. 그릴에 구운 치킨 샐러드로 주문할게요.
  - 웨이터: 훌륭한 선택입니다! 드레싱은 어떤 걸 원하시나요?
  - 당신: 네, 발사믹 비네그레트를 주세요.
  - 웨이터: 완벽합니다! 주문하겠습니다.


In [ ]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
# 스트리밍 출력
stream_response(answer)

In [38]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "미국에서 피자 주문"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:
**Customer:** Hi, I’d like to order a pizza for delivery, please.  
**Pizza Place:** Sure! What size would you like?  
**Customer:** I’ll have a large pizza.  
**Pizza Place:** Great! What toppings do you want?  
**Customer:** Can I get pepperoni and mushrooms?  
**Pizza Place:** Absolutely! Would you like any extra cheese or sides?  
**Customer:** Yes, I’d like extra cheese and a side of garlic bread, please.  
**Pizza Place:** Perfect! Can I have your address and phone number?  
**Customer:** Sure, it’s 123 Main Street, and my phone number is 555-1234.  
**Pizza Place:** Thank you! Your order will be delivered in about 30 minutes.  
**Customer:** Awesome, thank you!

- 한글 해석:
**고객:** 안녕하세요, 피자 배달 주문하고 싶어요.  
**피자 가게:** 물론이죠! 어떤 사이즈로 하시겠어요?  
**고객:** 큰 피자로 주세요.  
**피자 가게:** 좋습니다! 어떤 토핑을 원하시나요?  
**고객:** 페퍼로니와 버섯으로 해주세요.  
**피자 가게:** 알겠습니다! 추가 치즈나 사이드 메뉴는 필요하신가요?  
**고객:** 네, 추가 치즈와 마늘빵 하나 주세요.  
**피자 가게:** 완벽합니다! 주소와 전화번호를 알려주시겠어요?  
**고객:** 네, 123 메인 스트리트이고 제 전화번호는 555-1234입